In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pickle
import sys
from pathlib import Path

import lab.torch as B
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from wbml import plot

# torch.set_printoptions(precision=8, sci_mode=False, edgeitems=5)
torch.set_printoptions(profile='full') # 'short'

colors = sns.color_palette("bright")
sns.set_style("whitegrid")
sns.set_palette(colors)
matplotlib.rcParams["figure.dpi"] = 500  # for high quality, retina plots
matplotlib.use("Agg")
# from varz import Vars, namespace
from wbml import experiment, out, plot

/homes/mojb2/Thesis/GI-PVI/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
exp_dir = "results/07-23-19.24.09_seq-pvi-bank-1c-10g-1000l-nealprior-128b-0-05lr-10s-100m-gvi"
main_dir = os.path.dirname(os.path.abspath(os.getcwd()))

In [10]:
metrics_dir = os.path.join(main_dir, exp_dir, "metrics")

In [15]:
logs = {}
for log_name in os.listdir(metrics_dir):
    name = log_name.split('_')[0]
    logs[name] = pd.read_csv(os.path.join(metrics_dir, log_name))

In [17]:
logs['server']

,communications,iteration,train_mll,train_acc,test_mll,test_acc
0,0,0,-0.689368,0.883682,-0.689402,0.880336
1,2,1,-0.225419,0.901266,-0.233824,0.896483
2,4,2,-0.219282,0.904363,-0.227577,0.899248
3,6,3,-0.215091,0.906188,-0.223246,0.901239
4,8,4,-0.214032,0.906713,-0.222919,0.900575
5,10,5,-0.213125,0.907377,-0.221971,0.900133
6,12,6,-0.211779,0.907957,-0.220971,0.903229
7,14,7,-0.210333,0.909146,-0.219619,0.903451
8,16,8,-0.208476,0.909699,-0.218943,0.902234
9,18,9,-0.207463,0.909699,-0.217603,0.902787


In [8]:
vs_path = os.path.join(exp_dir, "model/_vs.pt")
vs_state_dict = torch.load(os.path.join(main_dir, vs_path))